In [0]:
-- Data Exploration
-- Count Rows (3732 rows)
SELECT COUNT(*) FROM product_page_info; 

-- Sample Data (First 10 rows visible)
SELECT * FROM product_page_info LIMIT 10;

-- Null Values (Zero values reported)
SELECT * FROM product_page_info
WHERE name IS NULL
OR 
Category IS NULL 
OR
mrp IS NULL 
OR
discountPercent IS NULL
OR
availableQuantity IS NULL 
OR
discountedSellingPrice IS NULL
OR
weightInGms IS NULL
OR
outOfStock IS NULL
OR
quantity IS NULL;

-- Different Product Categories (14 unique categories visible)
SELECT DISTINCT category
FROM product_page_info
ORDER BY category;

-- Product in-stock vs. out-of-stock (453 products out-of-stock; 3279 products in-stock)
SELECT outOfStock, COUNT(*)
FROM product_page_info
GROUP BY outOfStock;

-- Product names present multiple times
SELECT name, COUNT(*) AS `Number of SKUs`
FROM product_page_info
GROUP BY name
HAVING count(*) > 1
ORDER BY count(*) DESC;

-- Data Cleaning
-- Products with Price = 0 (1 row visible and is deleted)

SELECT * FROM product_page_info
WHERE mrp = 0 OR discountedSellingPrice = 0;

DELETE FROM product_page_info 
WHERE mrp = 0;

-- Convert Paise to Rupees
UPDATE product_page_info
SET mrp = mrp/100.0,
discountedSellingPrice = discountedSellingPrice/100.0;

-- Insight 1. Find the top 10 best-value products based on the discount percentage 
SELECT DISTINCT name, mrp, discountPercent
FROM product_page_info
ORDER BY discountPercent DESC
LIMIT 10;

-- Insight 2. Which product with high MRP are out-of-stock?
SELECT DISTINCT name, mrp
FROM product_page_info
WHERE outOfStock = TRUE AND mrp > 300
ORDER BY mrp DESC;

-- Insight 3. What is the estimated revenue from each product category? 
SELECT category,
SUM(discountedSellingPrice * availableQuantity) AS total_revenue
FROM product_page_info
GROUP BY category
ORDER BY total_revenue DESC;

-- Insight 4. Which products are priced over MRP 500 but discounted at less than 10%?
SELECT DISTINCT name, mrp, discountPercent
FROM product_page_info
WHERE mrp > 500 AND discountPercent < 10
ORDER BY mrp DESC, discountPercent DESC; 

-- Insight 5. Which top 5 product categories offer the highest average discount percentage?
SELECT category, AVG(discountPercent) AS avg_discount_percent
FROM product_page_info
GROUP BY category
ORDER BY avg_discount_percent DESC
LIMIT 5;

-- Insight 6. Locating price per gram for products above 100g and sort by best value
SELECT DISTINCT name, weightInGms, discountedSellingPrice, (discountedSellingPrice/weightInGms) AS price_per_gram
FROM product_page_info
WHERE weightInGms >= 100
ORDER BY price_per_gram DESC;

-- Insight 7. Group the products into categories like Low, Medium, Bulk based on product weights
SELECT DISTINCT name, weightInGms,
CASE WHEN weightInGms < 100 THEN 'Low'
     WHEN weightInGms BETWEEN 100 AND 5000 THEN 'Medium'
     ELSE 'Bulk'
     END AS weight_category
FROM product_page_info;

-- Insight 8. What is the Total Inventory Weight Per Category 
SELECT category,
SUM(weightInGms * availableQuantity) AS total_weight
FROM product_page_info
GROUP BY Category
ORDER BY total_weight DESC;
